In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Data loading and preparation
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

X_train = train_data['x'].values.reshape(-1, 1)
y_train = train_data['r'].values.reshape(-1, 1)
X_test = test_data['x'].values.reshape(-1, 1)
y_test = test_data['r'].values.reshape(-1, 1)

# Add bias term
X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
X_test = np.hstack((np.ones((X_test.shape[0], 1)), X_test))

# Single Layer Perceptron
class SingleLayerPerceptron:
    def __init__(self, input_size, learning_rate=0.01, epochs=1000):
        self.weights = np.random.randn(input_size, 1)
        self.learning_rate = learning_rate
        self.epochs = epochs

    def forward(self, X):
        return X @ self.weights

    def train(self, X, y):
        mse_history = []
        for epoch in range(self.epochs):
            y_pred = self.forward(X)
            error = y_pred - y
            mse = np.mean(error**2)
            mse_history.append(mse)
            gradient = X.T @ error / len(X)
            self.weights -= self.learning_rate * gradient
        return mse_history

# Multi Layer Perceptron
class MultiLayerPerceptron:
    def __init__(self, input_size, hidden_size, learning_rate=0.01, epochs=1000):
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2./(input_size + hidden_size))
        self.W2 = np.random.randn(hidden_size, 1) * np.sqrt(2./(hidden_size + 1))
        self.learning_rate = learning_rate
        self.epochs = epochs

    def tanh(self, x):
        return np.tanh(x)

    def tanh_derivative(self, x):
        return 1 - np.tanh(x)**2

    def forward(self, X):
        self.hidden_input = X @ self.W1
        self.hidden_output = self.tanh(self.hidden_input)
        self.output = self.hidden_output @ self.W2
        return self.output

    def train(self, X, y):
        mse_history = []
        for epoch in range(self.epochs):
            y_pred = self.forward(X)
            error = y_pred - y
            mse = np.mean(error**2)
            mse_history.append(mse)

            d_output = error / len(X)
            d_W2 = self.hidden_output.T @ d_output

            d_hidden = d_output @ self.W2.T * self.tanh_derivative(self.hidden_input)
            d_W1 = X.T @ d_hidden

            self.W2 -= self.learning_rate * d_W2
            self.W1 -= self.learning_rate * d_W1

        return mse_history

# Train and evaluate models
def main():
    # Initialize models
    slp = SingleLayerPerceptron(input_size=X_train.shape[1], learning_rate=0.01, epochs=1000)
    mlp_2 = MultiLayerPerceptron(input_size=X_train.shape[1], hidden_size=2, learning_rate=0.01, epochs=1000)
    mlp_4 = MultiLayerPerceptron(input_size=X_train.shape[1], hidden_size=4, learning_rate=0.01, epochs=1000)
    mlp_8 = MultiLayerPerceptron(input_size=X_train.shape[1], hidden_size=8, learning_rate=0.01, epochs=1000)

    # Train models
    slp_mse = slp.train(X_train, y_train)
    mlp_2_mse = mlp_2.train(X_train, y_train)
    mlp_4_mse = mlp_4.train(X_train, y_train)
    mlp_8_mse = mlp_8.train(X_train, y_train)

    # Test models
    def test_model(model, X, y):
        y_pred = model.forward(X)
        return np.mean((y_pred - y)**2)

    slp_test_mse = test_model(slp, X_test, y_test)
    mlp_2_test_mse = test_model(mlp_2, X_test, y_test)
    mlp_4_test_mse = test_model(mlp_4, X_test, y_test)
    mlp_8_test_mse = test_model(mlp_8, X_test, y_test)

    # Plot model outputs
    models = [slp, mlp_2, mlp_4, mlp_8]
    plot_model_outputs(models, X_train, y_train, X_test, y_test)

    # Plot learning curves
    train_mse = [slp_mse, mlp_2_mse, mlp_4_mse, mlp_8_mse]
    plot_learning_curves(train_mse)

    # Plot complexity vs error
    test_mse = [slp_test_mse, mlp_2_test_mse, mlp_4_test_mse, mlp_8_test_mse]
    plot_complexity_vs_error(test_mse)

    print("Training complete. Plots saved.")

def plot_model_outputs(models, X_train, y_train, X_test, y_test):
    plt.figure(figsize=(15, 10))
    titles = ['Single-Layer Perceptron', 'MLP with 2 Hidden Units',
              'MLP with 4 Hidden Units', 'MLP with 8 Hidden Units']

    sorted_idx = np.argsort(X_test[:, 1])
    X_test_sorted = X_test[sorted_idx]
    y_test_sorted = y_test[sorted_idx]

    for i, model in enumerate(models):
        plt.subplot(2, 2, i+1)
        plt.scatter(X_train[:, 1], y_train, color='blue', label='Training Data', alpha=0.5)
        plt.scatter(X_test[:, 1], y_test, color='green', label='Test Data', alpha=0.5)
        y_pred = model.forward(X_test_sorted)
        plt.plot(X_test_sorted[:, 1], y_pred, color='red', linewidth=2, label='Model Prediction')
        plt.title(titles[i])
        plt.xlabel('x')
        plt.ylabel('r')
        plt.legend()

    plt.tight_layout()
    plt.savefig('model_outputs.png')
    plt.close()

def plot_learning_curves(train_mse):
    plt.figure(figsize=(10, 6))
    labels = ['SLP', 'MLP-2', 'MLP-4', 'MLP-8']

    for mse, label in zip(train_mse, labels):
        plt.plot(mse, label=label)

    plt.title('Learning Curves (MSE vs Epochs)')
    plt.xlabel('Epochs')
    plt.ylabel('Mean Squared Error')
    plt.legend()
    plt.grid(True)
    plt.savefig('learning_curves.png')
    plt.close()

def plot_complexity_vs_error(test_mse):
    plt.figure(figsize=(8, 5))
    hidden_units = [0, 2, 4, 8]  # 0 for SLP

    plt.plot(hidden_units, test_mse, marker='o')
    plt.title('Model Complexity vs Test Error')
    plt.xlabel('Number of Hidden Units')
    plt.ylabel('Test MSE')
    plt.grid(True)
    plt.savefig('complexity_vs_error.png')
    plt.close()

if __name__ == "__main__":
    main()

Training complete. Plots saved.
